In [1]:
#instalando java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
#instalando spark
!pip install pyspark

In [3]:
#configurando variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

In [4]:
#iniciando SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("Contagem de palavras").getOrCreate()


In [7]:
#carregando o arquivo
caminho = '/content/README.md'
arquivo = spark.read.text(caminho)

#confirmando se o arquivo foi carregado corretamente
arquivo.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                   |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
from pyspark.sql.functions import explode, split, length, col

#separando as linhas por palavras
palavras = arquivo.select(explode(split(arquivo.value, '\s+')).alias('palavra'))

#filtrando palavras com 3 ou mais caracteres
palavras_filtradas = palavras.filter(length(col('palavra')) >= 3)

#contagem da ocorrência de cada palavra
contagem_palavras = palavras_filtradas.groupBy('palavra').count()

#ordenando as palavras por contagem decrescente
ordem = contagem_palavras.orderBy('count', ascending=False)

#exibindo as 15 palavras mais frequentes
ordem.show(15, truncate=False)

+--------+-----+
|palavra |count|
+--------+-----+
|para    |20   |
|com     |12   |
|que     |11   |
|![Clique|9    |
|filmes  |8    |
|bucket  |8    |
|dados   |7    |
|crime   |6    |
|função  |6    |
|dos     |6    |
|arquivo |5    |
|arquivos|5    |
|fim,    |4    |
|como    |4    |
|uma     |4    |
+--------+-----+
only showing top 15 rows

